## flow
    mbti - movie lens 합친 데이터 생성
    모델 불러오기
    애니어 그램 유사도 dataframe 불러오기
    mbti, engram, prefer_movie_ids 입력받기
    사용자가 선호하는 movie id로 부터 모델 추천 받기
    MBTI 필터링 -> 유사도 내림차순 정렬

## mbti - movie lens 합친 데이터 생성

In [66]:
import pandas as pd
import pickle

ml_title_df = pd.read_csv('/opt/ml/project/EDA/ml-25m/movies.csv')
ml_title_df = ml_title_df[['movieId','title']]

ml_title_df['title'] = ml_title_df['title'].str.lower()



def del_year(title):
    try:
        idx = title.index("(")
        return title[:idx].strip()
    except:
        return title.strip()

ml_title_df['title'] = ml_title_df['title'].apply(del_year)

print(ml_title_df.shape)

# ml_data 중복제거 전 중복을 확인해봤다.
# 3229의 영화가 같은 이름을 가지고있었다.
# 이름이 같더라도 장르가 다른 완전 다른 영화인 것 같다.
# mbti에 있는 영화중 해당 이름을 가진 영화수는 477개
# 방법 1) 그냥 drop_duplicate 해버린다. -> 데이터 질은 낮을 수 있지만 빠름
# 방법 2) 제목 뿐만아니라 다른정보도 일일이 확인을 해서 매칭시켜준다. -> 시간은 오래걸리지만 데이터질은 높음
# 지금은 방법 1)을 채택

ml_title_df = ml_title_df.drop_duplicates(subset='title',keep="first")

ml_title_df

ml_title_df.nunique()

mbti = pd.read_csv('/opt/ml/project/EDA/Personality_All_168000_rows.csv')

cat_dict={'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_1.png': 'n',
 'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_10.png': 'n',
 'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_11.png': 'gaming',
 'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_12.png': 'literature',
 'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_13.png': 'theatre', # 뮤지컬
 'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_14.png': 'n',
 'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_15.png': 'n',
 'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_16.png': 'n',
 'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_17.png': 'n',
 'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_18.png': 'n',
 'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_2.png': 'tv',
 'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_21.png': 'n',
 'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_22.png': 'n',
 'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_26.png': 'web_comics',
 'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_27.png': 'superheros',
 'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_28.png': 'n',
 'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_29.png': 'n',
 'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_3.png': 'movies', # 영화
 'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_5.png': 'n',
 'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_7.png': 'cartoons',
 'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_8.png': 'anime',
 'https://pdb-web-static.s3.us-east-2.amazonaws.com/category_icon/cat_9.png': 'comics'}

mbti.category = mbti.category.map(cat_dict)

(62423, 2)


In [67]:
ml_title_df

,movieId,title
0,1,toy story
1,2,jumanji
2,3,grumpier old men
3,4,waiting to exhale
4,5,father of the bride part ii
...,...,...
62417,209155,santosh subramaniam
62419,209159,window of the soul
62420,209163,bad poems
62421,209169,a girl thing


In [68]:
mbti_mv = mbti[mbti.category=='movies']

mbti_mv['title'] = mbti_mv['Contents'].str.lower()
def del_year(title):
    try:
        idx = title.index("(")
        return title[:idx].strip()
    except:
        return title.strip()

mbti_mv['title'] = mbti_mv['title'].apply(del_year)

/tmp/ipykernel_44530/1397074061.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mbti_mv['title'] = mbti_mv['Contents'].str.lower()
/tmp/ipykernel_44530/1397074061.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mbti_mv['title'] = mbti_mv['title'].apply(del_year)


In [69]:
mbti_mv

,Character,Contents,Votes,MBTI,Enneagram,category,img_src,Page,title
14,Moana Waialiki,Moana (2016),2.7k/477,ENFJ,7w6,movies,https://static1.personality-database.com/profi...,1,moana
16,Joy,Inside Out (2015),2.5k/358,ENFJ,7w6,movies,https://static1.personality-database.com/profi...,1,inside out
21,Judy Hopps,Zootopia (2016),2.3k/350,ENFJ,1w2,movies,https://static1.personality-database.com/profi...,1,zootopia
33,Elle Woods,Legally Blonde (2001),1.9k/227,ENFJ,3w2,movies,https://static1.personality-database.com/profi...,1,legally blonde
37,Neil Perry,Dead Poets Society (1989),1.7k/222,ENFJ,4w3,movies,https://static1.personality-database.com/profi...,1,dead poets society
...,...,...,...,...,...,...,...,...,...
167965,Berkeley Cole,Out of Africa (1985),6/0,INFJ,1w9,movies,https://static1.personality-database.com/profi...,210,out of africa
167972,The Girl,A Girl Walks Home Alone at Night (2014),6/0,INFJ,4w5,movies,https://static1.personality-database.com/profi...,210,a girl walks home alone at night
167975,Urzah,The Dark Crystal (1982),6/0,INFJ,9w1,movies,https://static1.personality-database.com/profi...,210,the dark crystal
167982,Muhsin Bey,Muhsin Bey,6/0,INFJ,1w2,movies,https://static1.personality-database.com/profi...,210,muhsin bey


In [70]:
mbti_mv_list = sorted(set(mbti_mv.title))
len(mbti_mv_list)

4185

In [71]:
mbti_mv.title.value_counts()

star wars                                  203
harry potter / wizarding world films       144
james bond film series                     106
toy story                                   77
transformers                                63
                                          ... 
batman begins                                1
how to talk to girls at parties              1
expelled                                     1
stormbreaker / alex rider: stormbreaker      1
muhsin bey                                   1
Name: title, Length: 4185, dtype: int64

In [72]:
ml_title_df.merge(mbti_mv, how='left', on='title')

,movieId,title,Character,Contents,Votes,MBTI,Enneagram,category,img_src,Page
0,1,toy story,Lots-o’-Huggin’ Bear (Lotso),Toy Story (Franchise),681/58,ENFJ,2w3,movies,https://static1.personality-database.com/profi...,3.0
1,1,toy story,Bo-Peep (Toy Story 4),Toy Story (Franchise),321/46,ENFJ,8w9,movies,https://static1.personality-database.com/profi...,7.0
2,1,toy story,Rib Tickles,Toy Story (Franchise),5/0,ENFJ,2w3,movies,https://static1.personality-database.com/profi...,206.0
3,1,toy story,Bonnie Anderson,Toy Story (Franchise),204/24,INFP,4w5,movies,https://static1.personality-database.com/profi...,15.0
4,1,toy story,Angel Kitty,Toy Story (Franchise),24/0,INFP,4w5,movies,https://static1.personality-database.com/profi...,103.0
...,...,...,...,...,...,...,...,...,...,...
67747,209155,santosh subramaniam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67748,209159,window of the soul,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67749,209163,bad poems,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67750,209169,a girl thing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
ml_title_df[ml_title_df.title.isin(mbti_mv_list)]

,movieId,title
0,1,toy story
1,2,jumanji
3,4,waiting to exhale
5,6,heat
6,7,sabrina
...,...,...
62273,208531,earthquake bird
62298,208655,the coldest game
62330,208773,i lost my body
62346,208813,noelle


In [74]:
mbti_mv2 = mbti_mv.merge(ml_title_df,on='title',how='left' )

In [80]:
mbti_mv2[mbti_mv2.movieId.notna()].Contents.value_counts()

Toy Story (Franchise)                     77
Transformers (Bayverse)                   63
Madagascar (Franchise)                    58
How to Train Your Dragon (Franchise)      55
Scream (Franchise)                        53
                                          ..
Love & Friendship (2016)                   1
Bajirao Mastani                            1
Black Mass (2015)                          1
Little Nemo: Adventures in Slumberland     1
Dark Waters (2019)                         1
Name: Contents, Length: 2737, dtype: int64

In [77]:
ml_title = ml_title_df.copy()
with open('mbti_ml_match_dict.pkl', 'rb') as handle:
    match_dict = pickle.load(handle)
ml_title = ml_title.dropna(axis=0)
#{'mbti_title':'ml_title'} -> {'ml_title':'mbti_title'}
match_dict2 = {v:k for k, v in match_dict.items()}

ml_title['mbti_title'] = ml_title['title'].map(match_dict2)
ml_title = ml_title.dropna()
ml_title = ml_title.drop_duplicates(subset='title', keep='first')
ml_title

,movieId,title,mbti_title
31,32,twelve monkeys,12 monkeys / twelve monkeys
43,44,mortal kombat,mortal kombat films
49,50,"usual suspects, the",the usual suspects
72,73,"misérables, les",les miserables
103,105,"bridges of madison county, the",the bridges of madison county
...,...,...,...
60434,203298,checkered ninja,ternet ninja
60626,203745,turma da mônica: laços,turma da mônica
61007,204702,"where'd you go, bernadette","where’d you go, bernadette"
61144,205076,downton abbey,downtown abbey


In [86]:
ml_title2=ml_title[['mbti_title','movieId']]
ml_title2

,mbti_title,movieId
31,12 monkeys / twelve monkeys,32
43,mortal kombat films,44
49,the usual suspects,50
72,les miserables,73
103,the bridges of madison county,105
...,...,...
60434,ternet ninja,203298
60626,turma da mônica,203745
61007,"where’d you go, bernadette",204702
61144,downtown abbey,205076


In [84]:
mbti_mv2[mbti_mv2.title.str.contains('twelve monkeys')]

,Character,Contents,Votes,MBTI,Enneagram,category,img_src,Page,title,movieId
1530,Kathryn Railly,12 Monkeys / Twelve Monkeys (1995),37/1,INFP,5w6,movies,https://static1.personality-database.com/profi...,72,12 monkeys / twelve monkeys,NaN
3509,James Cole,12 Monkeys / Twelve Monkeys (1995),53/5,INTP,6w5,movies,https://static1.personality-database.com/profi...,43,12 monkeys / twelve monkeys,NaN
4565,Jeffrey Goines,12 Monkeys / Twelve Monkeys (1995),45/2,ENTP,8w7,movies,https://static1.personality-database.com/profi...,61,12 monkeys / twelve monkeys,NaN
4570,Jeffrey Goines,12 Monkeys / Twelve Monkeys (1995),45/2,ENTP,8w7,movies,https://static1.personality-database.com/profi...,63,12 monkeys / twelve monkeys,NaN
5316,Dr. Peters,12 Monkeys / Twelve Monkeys (1995),9/0,ENTP,XwX,movies,https://pdb-web-static.s3.us-east-2.amazonaws....,202,12 monkeys / twelve monkeys,NaN
15847,Jose,12 Monkeys / Twelve Monkeys (1995),7/0,ISFP,XwX,movies,https://pdb-web-static.s3.us-east-2.amazonaws....,204,12 monkeys / twelve monkeys,NaN


In [90]:
ml_title2_mvid = mbti_mv2[['title']].merge(ml_title2, left_on='title', right_on='mbti_title', how='left').movieId

In [93]:
mbti_mv2.movieId = mbti_mv2.movieId.fillna(ml_title2_mvid)

In [96]:
mbti_mv2[mbti_mv2.movieId.notna()].Contents.value_counts()

Star Wars                                  203
Harry Potter / Wizarding World Films       144
Toy Story (Franchise)                       77
Transformers (Bayverse)                     63
Madagascar (Franchise)                      58
                                          ... 
The Artist (2011)                            1
Working Girl                                 1
Because of Winn-Dixie                        1
On The Edge                                  1
A Girl Walks Home Alone at Night (2014)      1
Name: Contents, Length: 3348, dtype: int64

In [99]:
mbti_mv3 = mbti_mv2[mbti_mv2.movieId.notna()]
mbti_mv3.shape

(15926, 10)

In [101]:
mbti_mv3.movieId.value_counts()

184593.0    203
69844.0     144
1.0          77
53996.0      63
33615.0      58
           ... 
67997.0       1
45503.0       1
14.0          1
153648.0      1
118880.0      1
Name: movieId, Length: 3229, dtype: int64

In [157]:
mbti_mv3.to_csv('MBTI_merge_movie_id_3229_rows.csv',index=False)

In [104]:
mbti_mv3

,Character,Contents,Votes,MBTI,Enneagram,category,img_src,Page,title,movieId
0,Moana Waialiki,Moana (2016),2.7k/477,ENFJ,7w6,movies,https://static1.personality-database.com/profi...,1,moana,73141.0
1,Joy,Inside Out (2015),2.5k/358,ENFJ,7w6,movies,https://static1.personality-database.com/profi...,1,inside out,122470.0
2,Judy Hopps,Zootopia (2016),2.3k/350,ENFJ,1w2,movies,https://static1.personality-database.com/profi...,1,zootopia,152081.0
3,Elle Woods,Legally Blonde (2001),1.9k/227,ENFJ,3w2,movies,https://static1.personality-database.com/profi...,1,legally blonde,4447.0
4,Neil Perry,Dead Poets Society (1989),1.7k/222,ENFJ,4w3,movies,https://static1.personality-database.com/profi...,1,dead poets society,1246.0
...,...,...,...,...,...,...,...,...,...,...
19826,Diana Reiter,Schindler's List (1993),6/0,INFJ,5w6,movies,https://static1.personality-database.com/profi...,210,schindler's list,527.0
19828,Berkeley Cole,Out of Africa (1985),6/0,INFJ,1w9,movies,https://static1.personality-database.com/profi...,210,out of africa,1959.0
19829,The Girl,A Girl Walks Home Alone at Night (2014),6/0,INFJ,4w5,movies,https://static1.personality-database.com/profi...,210,a girl walks home alone at night,118880.0
19830,Urzah,The Dark Crystal (1982),6/0,INFJ,9w1,movies,https://static1.personality-database.com/profi...,210,the dark crystal,91077.0


## 모델, 애니어 그램 유사도 dataframe 불러오기

In [158]:
import sys
sys.path.append('../interaction_model/')
from inference import Inference

model_path='../interaction_model/similarity'
Inference(model_path, 1, 10)

[780, 648, 736, 32, 733, 608, 480, 2355, 588]

In [111]:
en_sim = pd.read_csv('enneagram_similarity.csv')
en_sim.index = en_sim.columns
en_sim[['1w9']].to_dict()['1w9']

{'1w9': 1.0000000000000002,
 '1w2': 0.4802285080172036,
 '2w1': 0.1111942598655678,
 '2w3': 0.0469149991354468,
 '3w2': 0.2168643887422373,
 '3w4': 0.3082825863926258,
 '4w3': 0.0587421277813055,
 '4w5': 0.1185341861968765,
 '5w4': 0.3378192760628618,
 '5w6': 0.5097201729093104,
 '6w5': 0.719973601967801,
 '6w7': 0.1010946909305798,
 '7w6': 0.0390903461112397,
 '7w8': 0.0438712262186794,
 '8w7': 0.2632703411832457,
 '8w9': 0.340520602302011,
 '9w8': 0.0191522747113997,
 '9w1': 0.4025998141307324}

In [112]:
def get_en_sim(en):
    return en_sim[[en]].to_dict()[en]

## mbti, engram, prefer_movie_ids 입력받기

In [120]:
prefer_movies=['Avatar (Franchise)','The Hunger Games (Franchise)','La La Land (2016)','Moana (2016)','Dune (2021)','Good Will Hunting (1997)','Titanic (1997)',]
len(prefer_movies)
print(prefer_movies)

['Avatar (Franchise)', 'The Hunger Games (Franchise)', 'La La Land (2016)', 'Moana (2016)', 'Dune (2021)', 'Good Will Hunting (1997)', 'Titanic (1997)']


In [124]:
prefer_movie_ids = sorted(set(mbti_mv3[mbti_mv3.Contents.isin(prefer_movies)].movieId))
prefer_movie_ids

[1704.0, 1721.0, 2021.0, 72998.0, 73141.0, 91500.0, 164909.0]

In [125]:
mbti="INTJ"
engram="3w2"

## 사용자가 선호하는 movie id로 부터 모델 추천 받기

In [134]:
recommend_movieIds=[]
for mid in prefer_movie_ids:
    recommend_movieIds.extend(Inference(model_path, int(mid), 10)) 
len(recommend_movieIds)

63

In [135]:
recommend_movieIds = list(set(recommend_movieIds))
len(recommend_movieIds)

59

In [168]:
mbti_mv3.Contents.nunique()

3348

In [167]:
mbti_mv3

,Character,Contents,Votes,MBTI,Enneagram,category,img_src,Page,title,movieId
0,Moana Waialiki,Moana (2016),2.7k/477,ENFJ,7w6,movies,https://static1.personality-database.com/profi...,1,moana,73141.0
1,Joy,Inside Out (2015),2.5k/358,ENFJ,7w6,movies,https://static1.personality-database.com/profi...,1,inside out,122470.0
2,Judy Hopps,Zootopia (2016),2.3k/350,ENFJ,1w2,movies,https://static1.personality-database.com/profi...,1,zootopia,152081.0
3,Elle Woods,Legally Blonde (2001),1.9k/227,ENFJ,3w2,movies,https://static1.personality-database.com/profi...,1,legally blonde,4447.0
4,Neil Perry,Dead Poets Society (1989),1.7k/222,ENFJ,4w3,movies,https://static1.personality-database.com/profi...,1,dead poets society,1246.0
...,...,...,...,...,...,...,...,...,...,...
19826,Diana Reiter,Schindler's List (1993),6/0,INFJ,5w6,movies,https://static1.personality-database.com/profi...,210,schindler's list,527.0
19828,Berkeley Cole,Out of Africa (1985),6/0,INFJ,1w9,movies,https://static1.personality-database.com/profi...,210,out of africa,1959.0
19829,The Girl,A Girl Walks Home Alone at Night (2014),6/0,INFJ,4w5,movies,https://static1.personality-database.com/profi...,210,a girl walks home alone at night,118880.0
19830,Urzah,The Dark Crystal (1982),6/0,INFJ,9w1,movies,https://static1.personality-database.com/profi...,210,the dark crystal,91077.0


## MBTI 필터링 -> 유사도 내림차순 정렬

In [137]:
recommend_chars = mbti_mv3[mbti_mv3.movieId.isin(recommend_movieIds)]
recommend_chars

,Character,Contents,Votes,MBTI,Enneagram,category,img_src,Page,title,movieId
11,Finnick Odair,The Hunger Games (Franchise),753/40,ENFJ,2w3,movies,https://static1.personality-database.com/profi...,3,the hunger games,91500.0
24,Commodus,Gladiator (2000),359/17,ENFJ,4w3,movies,https://static1.personality-database.com/profi...,6,gladiator,3578.0
52,Bridget von Hammersmark,Inglourious Basterds (2009),193/10,ENFJ,3w2,movies,https://static1.personality-database.com/profi...,11,inglourious basterds,68157.0
74,Otto,Despicable Me (Franchise),145/15,ENFJ,7w6,movies,https://static1.personality-database.com/profi...,15,despicable me,79091.0
82,Skylar,Good Will Hunting (1997),133/4,ENFJ,2w1,movies,https://static1.personality-database.com/profi...,16,good will hunting,1704.0
...,...,...,...,...,...,...,...,...,...,...
19761,Lucy,Lion (2016),7/0,INFJ,5w6,movies,https://static1.personality-database.com/profi...,198,lion,165551.0
19766,Lucy,Lion (2016),7/0,INFJ,5w6,movies,https://static1.personality-database.com/profi...,199,lion,165551.0
19787,Lucy,Lion (2016),7/0,INFJ,5w6,movies,https://static1.personality-database.com/profi...,202,lion,165551.0
19798,Sue Brierley,Lion (2016),7/0,INFJ,2w1,movies,https://static1.personality-database.com/profi...,205,lion,165551.0


In [160]:
mbti_same_rec = recommend_chars[recommend_chars.MBTI==mbti]
print(mbti_same_rec.shape)
mbti_same_rec

(41, 10)


,Character,Contents,Votes,MBTI,Enneagram,category,img_src,Page,title,movieId
5373,Gandalf the Grey,The Lord of the Rings (Film Trilogy),2k/273,INTJ,5w6,movies,https://static1.personality-database.com/profi...,2,the lord of the rings,2116.0
5375,V,V for Vendetta (2005),1.9k/185,INTJ,5w4,movies,https://static1.personality-database.com/profi...,2,v for vendetta,44191.0
5377,Felonius Gru,Despicable Me (Franchise),1.6k/127,INTJ,3w4,movies,https://static1.personality-database.com/profi...,2,despicable me,79091.0
5388,Saruman the White,The Lord of the Rings (Film Trilogy),678/37,INTJ,5w6,movies,https://static1.personality-database.com/profi...,5,the lord of the rings,2116.0
5390,Shosanna Dreyfus,Inglourious Basterds (2009),628/39,INTJ,1w9,movies,https://static1.personality-database.com/profi...,5,inglourious basterds,68157.0
5400,Elrond,The Lord of the Rings (Film Trilogy),383/11,INTJ,1w2,movies,https://static1.personality-database.com/profi...,8,the lord of the rings,2116.0
5407,Christof,The Truman Show (1998),304/14,INTJ,3w4,movies,https://static1.personality-database.com/profi...,10,the truman show,1682.0
5417,"Robert Callaghan ""Yokai""",Big Hero 6 (2014),241/14,INTJ,8w9,movies,https://static1.personality-database.com/profi...,12,big hero 6,115617.0
5425,Gandalf the White,The Lord of the Rings (Film Trilogy),218/16,INTJ,1w9,movies,https://static1.personality-database.com/profi...,14,the lord of the rings,2116.0
5432,Professor James Moriarty,Sherlock Holmes,201/11,INTJ,5w6,movies,https://static1.personality-database.com/profi...,15,sherlock holmes,73017.0


In [142]:
engram_dict = get_en_sim(engram)
engram_dict

{'1w9': 0.2168643887422373,
 '1w2': 0.4486612282123909,
 '2w1': 0.454278981632048,
 '2w3': 0.5068529020255618,
 '3w2': 1.0,
 '3w4': 0.7610787374418372,
 '4w3': 0.3622404705489577,
 '4w5': 0.1732080282376144,
 '5w4': 0.1037384154443334,
 '5w6': 0.0628633453495011,
 '6w5': 0.2141641942074714,
 '6w7': 0.5112711478291062,
 '7w6': 0.2886293824107214,
 '7w8': 0.3226294742133307,
 '8w7': 0.7561504616639425,
 '8w9': 0.7532938288392081,
 '9w8': 0.2490405473718453,
 '9w1': 0.2661726665332223}

In [162]:
import warnings
warnings.filterwarnings('ignore')

In [165]:
mbti_same_rec['Enneagram_sim'] = mbti_same_rec.Enneagram.map(engram_dict)

In [166]:
rec_result = mbti_same_rec.sort_values('Enneagram_sim', ascending=False)
rec_result

,Character,Contents,Votes,MBTI,Enneagram,category,img_src,Page,title,movieId,Enneagram_sim
5377,Felonius Gru,Despicable Me (Franchise),1.6k/127,INTJ,3w4,movies,https://static1.personality-database.com/profi...,2.0,despicable me,79091.0,0.761079
5407,Christof,The Truman Show (1998),304/14,INTJ,3w4,movies,https://static1.personality-database.com/profi...,10.0,the truman show,1682.0,0.761079
5417,"Robert Callaghan ""Yokai""",Big Hero 6 (2014),241/14,INTJ,8w9,movies,https://static1.personality-database.com/profi...,12.0,big hero 6,115617.0,0.753294
5983,John Reid,Bohemian Rhapsody (2018),14/1,INTJ,6w7,movies,https://static1.personality-database.com/profi...,127.0,bohemian rhapsody,192803.0,0.511271
5734,TRON,Tron (Series),28/1,INTJ,1w2,movies,https://static1.personality-database.com/profi...,78.0,tron,2105.0,0.448661
5400,Elrond,The Lord of the Rings (Film Trilogy),383/11,INTJ,1w2,movies,https://static1.personality-database.com/profi...,8.0,the lord of the rings,2116.0,0.448661
5935,Agent O,Men in Black,16/0,INTJ,1w9,movies,https://static1.personality-database.com/profi...,118.0,men in black,1580.0,0.216864
5482,Ben Cash,Captain Fantastic (2016),109/7,INTJ,1w9,movies,https://static1.personality-database.com/profi...,26.0,captain fantastic,158966.0,0.216864
5834,Dr. Arthur Neuman,The Mask (1994),21/0,INTJ,1w9,movies,https://static1.personality-database.com/profi...,97.0,the mask,124476.0,0.216864
5491,Paul Finch,American Pie,102/3,INTJ,1w9,movies,https://static1.personality-database.com/profi...,27.0,american pie,2706.0,0.216864


In [153]:
result = rec_result.drop_duplicates(subset=['Character', 'Contents'])[:10]
result

,Character,Contents,Votes,MBTI,Enneagram,category,img_src,Page,title,movieId,Enneagram_sim
5377,Felonius Gru,Despicable Me (Franchise),1.6k/127,INTJ,3w4,movies,https://static1.personality-database.com/profi...,2,despicable me,79091.0,0.761079
5407,Christof,The Truman Show (1998),304/14,INTJ,3w4,movies,https://static1.personality-database.com/profi...,10,the truman show,1682.0,0.761079
5417,"Robert Callaghan ""Yokai""",Big Hero 6 (2014),241/14,INTJ,8w9,movies,https://static1.personality-database.com/profi...,12,big hero 6,115617.0,0.753294
5983,John Reid,Bohemian Rhapsody (2018),14/1,INTJ,6w7,movies,https://static1.personality-database.com/profi...,127,bohemian rhapsody,192803.0,0.511271
5734,TRON,Tron (Series),28/1,INTJ,1w2,movies,https://static1.personality-database.com/profi...,78,tron,2105.0,0.448661
5400,Elrond,The Lord of the Rings (Film Trilogy),383/11,INTJ,1w2,movies,https://static1.personality-database.com/profi...,8,the lord of the rings,2116.0,0.448661
5935,Agent O,Men in Black,16/0,INTJ,1w9,movies,https://static1.personality-database.com/profi...,118,men in black,1580.0,0.216864
5482,Ben Cash,Captain Fantastic (2016),109/7,INTJ,1w9,movies,https://static1.personality-database.com/profi...,26,captain fantastic,158966.0,0.216864
5834,Dr. Arthur Neuman,The Mask (1994),21/0,INTJ,1w9,movies,https://static1.personality-database.com/profi...,97,the mask,124476.0,0.216864
5491,Paul Finch,American Pie,102/3,INTJ,1w9,movies,https://static1.personality-database.com/profi...,27,american pie,2706.0,0.216864


In [155]:
cols=['Character','Contents','img_src','Enneagram_sim']
result[cols]

,Character,Contents,img_src,Enneagram_sim
5377,Felonius Gru,Despicable Me (Franchise),https://static1.personality-database.com/profi...,0.761079
5407,Christof,The Truman Show (1998),https://static1.personality-database.com/profi...,0.761079
5417,"Robert Callaghan ""Yokai""",Big Hero 6 (2014),https://static1.personality-database.com/profi...,0.753294
5983,John Reid,Bohemian Rhapsody (2018),https://static1.personality-database.com/profi...,0.511271
5734,TRON,Tron (Series),https://static1.personality-database.com/profi...,0.448661
5400,Elrond,The Lord of the Rings (Film Trilogy),https://static1.personality-database.com/profi...,0.448661
5935,Agent O,Men in Black,https://static1.personality-database.com/profi...,0.216864
5482,Ben Cash,Captain Fantastic (2016),https://static1.personality-database.com/profi...,0.216864
5834,Dr. Arthur Neuman,The Mask (1994),https://static1.personality-database.com/profi...,0.216864
5491,Paul Finch,American Pie,https://static1.personality-database.com/profi...,0.216864


In [156]:
result[cols].to_dict(orient='records')

[{'Character': 'Felonius Gru',
  'Contents': 'Despicable Me (Franchise)',
  'img_src': 'https://static1.personality-database.com/profile_images/f91fdbe3a72145c68cc0933fd66e5e65.png',
  'Enneagram_sim': 0.7610787374418372},
 {'Character': 'Christof',
  'Contents': 'The Truman Show (1998)',
  'img_src': 'https://static1.personality-database.com/profile_images/5d29da97f7284feb83c4b71d8cc9c8a8.png',
  'Enneagram_sim': 0.7610787374418372},
 {'Character': 'Robert Callaghan "Yokai"',
  'Contents': 'Big Hero 6 (2014)',
  'img_src': 'https://static1.personality-database.com/profile_images/a591bb9cc64d4a49baf5d052ccbc75fc.png',
  'Enneagram_sim': 0.7532938288392081},
 {'Character': 'John Reid',
  'Contents': 'Bohemian Rhapsody (2018)',
  'img_src': 'https://static1.personality-database.com/profile_images/51a7135ed3ad4b02887b5605f56acd05.png',
  'Enneagram_sim': 0.5112711478291062},
 {'Character': 'TRON',
  'Contents': 'Tron (Series)',
  'img_src': 'https://static1.personality-database.com/profil